In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
data_df = pd.read_csv(r"C:\Coding\Python learnen materials\Projects\RicetypeDataset\riceClassification.csv")


In [7]:
data_df.dropna(inplace=True)

In [ ]:
data_df.drop(columns='id', inplace=True)

KeyError: "['id'] not found in axis"

In [12]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [ ]:
#Basic cleaning of data
original_data_df = data_df.copy()
for column in data_df.columns:
    data_df[column] = data_df[column]/data_df[column].abs().max()

data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [17]:
#Organising the data into features and label sets
x = np.array(data_df.iloc[: , :-1])
y = np.array(data_df.iloc[: , -1])

#Splitting the dataset into train, val and test set

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)
x_test,x_val, y_test,y_val = train_test_split(x_test,y_test, test_size=0.5)

In [18]:
#Dataset class
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype= torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [ ]:
#loading the dataset
training_data = dataset(x_train, y_train)
validation_data = dataset(x_val ,y_val)
test_data = dataset(x_test, y_test)

In [20]:
#loading the dataloader
training_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
validation_datalaoder = DataLoader(validation_data, batch_size=32, shuffle=True)
test_data = DataLoader(test_data, batch_size=32, shuffle=True)

In [24]:
#Desigining model

hidden_neurons = 10
class ricemodel(nn.Module):
    def __init__(self):
        super(ricemodel, self). __init__()
        self.input_layer = nn.Linear(x.shape[1], hidden_neurons)
        self.linear = nn.Linear(hidden_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.input_layer(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

model = ricemodel().to(device)

In [26]:
loss = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 1e-3)


In [ ]:
#training loop
